In [1]:
import numpy as np
from matplotlib import pyplot as plt

import os
import pickle
from time import time
from sklearn.decomposition import PCA

import george
from george import kernels
from scipy.optimize import minimize

import lhsmdu
import os
import pdspy.modeling as modeling

from matplotlib.animation import FuncAnimation

%matplotlib inline

In [2]:
param_names = ["Tstar","logL_star","logM_disk","logR_disk","h_0","logR_in",\
          "gamma","beta","logM_env","logR_env","f_cav","ksi","loga_max","p","incl"]

bases=[4000.0, 1.0, -5.0, 1.5, 0.255, 0.75, 1.0, 1.25, -5.0, 3.25, 0.5, 1.0, 2.5, 3.5, 45.0]
ranges = [[3000.,5000.], [-1,3.],[-8.,-2.], [0.,3.],[0.01,0.5],[-1.,2.5],[0.0,2.0],\
        [0.5,2.0],[-8.,-2.],[2.5,4.], [0.,1.], [0.5,1.5],[0.,5.],[2.5,4.5],[0.,90.]]

dictionary=np.load("../grid_metadata/dictionary.npy")

# get all model data
with open ('../grid_metadata/cubefull.txt', 'rb') as fp:
    cube = np.array(pickle.load(fp))[:,100:500]
with open ('../grid_metadata/cubefull.txt', 'rb') as fp:
    nancube = np.array(pickle.load(fp))[:,100:500]
    
# x values (wavelengths) - 500 values, in normal space
with open ('../grid_metadata/xvals.txt', 'rb') as fp:
    xvals = pickle.load(fp)[100:500]

# fix -infs: powerlaw cutoff
for i in range(len(cube)):
    if -np.inf in cube[i]:
        a = cube[i].tolist()
        a.reverse()
        ind = len(a)-a.index(-np.inf)
        x1 = xvals[ind]
        y1 = cube[i][ind]
        for j in range(ind):
            cube[i][j]=(100*(np.log10(xvals[j]/x1)))+y1
            
# nan cutoff for means            
nancube[nancube<-20]=np.nan

# subtracting from the seds each sample mean
seds_msub = cube - np.nanmean(nancube,axis=1)[:,np.newaxis]

t0 = time()
pca = PCA(n_components=40).fit(seds_msub)
print("done in %0.3fs" % (time() - t0))

eigenseds=np.array(pca.components_)

fitdata=[]

for i in range(len(cube)):
    modeldata=[]
    coeffs=pca.transform(seds_msub[i].reshape(1,-1))
    for k in range(15):
        modeldata.append(coeffs[0][k])
    fitdata.append(modeldata)
    
def load_pcwpar(weight):
    p=[]
    w=[]
    m=[]
    for i in range(len(cube)):
        pars=[]
        for j in range(len(param_names)):
            pars.append(dictionary[i][param_names[j]])
        p.append(pars)
        w.append(fitdata[i][weight])
        m.append(np.nanmean(nancube,axis=1)[i])
    
    return p,w,m

def load_pcwpar_all():
    p=[]
    w=[]
    m=[]
    for i in range(len(cube)):
        pars=[]
        for j in range(len(param_names)):
            pars.append(dictionary[i][param_names[j]])
        p.append(pars)
        weights=[]
        for k in range(15):
            weights.append(fitdata[i][k])
        w.append(weights)
        m.append(np.nanmean(nancube,axis=1)[i])
    return p,w,m

done in 0.449s


In [3]:
Xs,ws,ms=load_pcwpar_all()
ws=np.transpose(ws)
ws1=np.ndarray.tolist(ws)
ws1.append(ms)
ws=ws1

yerrs=[]
for i in range(16):
    yerrs.append([x*0.01 for x in ws[i]])

In [4]:
t0=time()
gps=[]
for i in range(len(ws)):
    kernel = np.var(ws[i])*kernels.ExpSquaredKernel(10000**2,ndim=15,axes=0)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=1)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=2)*\
        kernels.ExpSquaredKernel(0.8**2,ndim=15,axes=3)*\
        kernels.ExpSquaredKernel(0.08**2,ndim=15,axes=4)*\
        kernels.ExpSquaredKernel(0.5**2,ndim=15,axes=5)*\
        kernels.PolynomialKernel(1.7,15,ndim=15,axes=6)*\
        kernels.ExpSquaredKernel(0.2**2,ndim=15,axes=7)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=8)*\
        kernels.ExpSquaredKernel(0.2**2,ndim=15,axes=9)*\
        kernels.ExpSquaredKernel(0.25**2,ndim=15,axes=10)*\
        kernels.ExpSquaredKernel(0.25**2,ndim=15,axes=11)*\
        kernels.ExpSquaredKernel(2**2,ndim=15,axes=12)*\
        kernels.ExpSquaredKernel(3**2,ndim=15,axes=13)*\
        kernels.ExpSquaredKernel(5**2,ndim=15,axes=14) 

    #11, ksi - not smooth. may need to change kernel
    #6 is gamma - polynomial kernel for tail
    # change lengthscale for 7 to ~4 or 5

    gps.append(george.GP(kernel))
    gps[i].compute(Xs,yerrs[i])
print("done in %0.3fs" % (time() - t0))

done in 61.858s


In [6]:
preds=[]
for i in range(16):
    pred, pred_var = gps[i].predict(ws[i], Xs, return_var=True)
    preds.append(pred)

In [8]:
np.array(preds).shape

(16, 3850)

In [15]:
li=[]
for i in range(5):
    rec=np.dot(np.array(preds)[:,i][0:15],eigenseds[0:15]) + pca.mean_ + np.array(preds)[:,i][15]
    li.append(rec)

In [18]:
np.array(preds).shape

(16, 3850)

In [22]:
np.dot(np.array(preds)[:][0:15],eigenseds[0:15][:,np.newaxis])# + pca.mean_ + np.array(preds)[15]

ValueError: shapes (15,3850) and (15,1,400) not aligned: 3850 (dim 1) != 1 (dim 1)

In [26]:
a=np.dot(np.array(preds)[:,0][0:15],eigenseds[0:15])
np.array(a).shape

(400,)

In [30]:
np.printoptions()
np.array(preds)[:,0][0:15]

array([-1.41440535e+00,  1.95445881e+01, -1.57513494e+01,  4.12491179e+00,
       -2.52375650e+00,  8.21880817e+00,  1.71290946e+00,  3.08529305e+00,
       -1.37819755e+00, -1.61949480e+00,  3.31167400e-01,  1.16531432e-01,
       -1.32742524e-02,  4.03375328e-01, -2.92652428e-01])